# Section 8.2: Time Matrix Multiply

Adapted from: "Guide to Fortran 2008 Programming" by Walter S. Brainerd (Springer 2015)

## Program to demonstrate the performance improvement of using matmul over do loops.

```fortran
program time_matrix_multiply

    ! Compare times of the matmul intrinsic cs DO loops

    implicit none
    integer, parameter :: n = 10
    real, dimension (n, n) :: a, b, c1, c2
    character(len=8) :: date
    real :: start_time_1, stop_time_1, start_time_2, stop_time_2
    real :: total_time_1, total_time_2
    integer :: i, j, k
    character(len=*), parameter :: form = "(t2, a, f0.10, a)"

    ! Get date to print on report

    call date_and_time(date = date)

    print *, "Timing report dated: " // date(1:4) &
        // "-" // date(5:6) // "-" // date(7:8)

    call random_seed()
    call random_number(a)
    call random_number(b)
    call cpu_time(start_time_1)
   
    ! Lines below added for effect
    ! The matrices of random values will be printed to the screen
    write (*, "(a)") "Matrix A"
    write (*, "(10f10.3)") (a(i,:), i=1,n)

    print *
    write (*, "(a)") "Matrix B"
    write (*, "(10f10.3)") (b(i,:), i=1,n)

    c1 = 0

    do k = 1, n
        do j = 1, n
            do i = 1, n
                c1(i, j) = c1(i, j) + a(i, k) * b(k, j)
            end do
        end do
    end do

    call cpu_time(stop_time_1)

    total_time_1 = stop_time_1 - start_time_1

    print *

    write (*, "(a)") "Matrix C1 is AxB using loops."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    call cpu_time(start_time_2)
    c2 = matmul(a, b)
    call cpu_time(stop_time_2)

    total_time_2 = stop_time_2 - start_time_2

    print *

    write (*, "(a)") "Matrix C2 is AxB using matmul."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    print *

    write (*, form) "Time of Do loop version is: ", total_time_1, " seconds."
    write (*, form) "Time of matmul version is: ", total_time_2, " seconds."

    print *

    if (any(abs(c1-c2) > 1.0e-4)) then
        write (*,  "(a)") "There are significantly different values between the matrices."
    else
        write (*,  "(a)") "The results are approximately the same."
    end if

    print *
    write (*, "(a, f10.3, a)") "The speedup ratio is: ", total_time_1/total_time_2, "x"

end program time_matrix_multiply
```

The above program is compiled and run using Fortran Package Manager (fpm):

In [1]:
import os
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_8_2_Time_Matrix_Multiply"

In [3]:
os.chdir(code_dir)

In [4]:
build_status = os.system("fpm build 2>/dev/null")

 + mkdir -p build/dependencies
[  0%] section_8_2_time_matrix_multip
[ 50%] section_8_2_time_matrix_multip  done.
[ 50%] Section_8_2_Time_Matrix_Multip
[100%] Section_8_2_Time_Matrix_Multip  done.
[100%] Project compiled successfully.


In [5]:
exec_status = os.system("fpm run 2>/dev/null")

 Timing report dated: 2022-12-20
Matrix A
     0.335     0.334     0.925     0.901     0.314     0.498     0.399     0.349     0.945     0.926
     0.267     0.491     0.305     0.005     0.207     0.451     0.095     0.224     0.068     0.891
     0.189     0.517     0.112     0.872     0.107     0.037     0.040     0.128     0.246     0.836
     0.635     0.559     0.513     0.759     0.016     0.445     0.201     0.449     0.122     0.136
     0.045     0.680     0.898     0.364     0.572     0.526     0.409     0.596     0.687     0.924
     0.436     0.766     0.616     0.276     0.626     0.001     0.881     0.627     0.581     0.194
     0.764     0.159     0.745     0.916     0.938     0.769     0.541     0.369     0.472     0.482
     0.038     0.459     0.106     0.249     0.888     0.455     0.410     0.098     0.613     0.720
     0.198     0.181     0.575     0.393     0.363     0.390     0.239     0.427     0.690     0.599
     0.391     0.702     0.648     0.133     0.78